<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - ALS MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [3]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als
from lenskit.algorithms.als import BiasedMF
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [4]:
def model_trainer(features,reg):
    all_recs = []
    test_data = []
    
    features=int(features)
    als = BiasedMF(features=features, iterations=100, reg=reg, damping=5, bias=True,  progress=None) # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('ALS', als, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.recip_rank)
    results = rla.compute(all_recs, test_data)
                          
    return results["recip_rank"].mean()

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    features = params['features']
    reg = params['reg']
    metric = model_trainer(features,reg)
    print([features,reg,metric],',')
    return {'loss': -metric, 'status': STATUS_OK}

space={'features': hp.uniform('features', 1, 100),
       'reg': hp.uniform('reg',0.01,0.4)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[27.149627715381794, 0.1858475031658863, 0.1797702155028539]                                                           
,                                                                                                                      
[67.0080586166762, 0.01614439296735901, 0.18812390955620187]                                                           
,                                                                                                                      
[99.60546834896662, 0.25492718310042795, 0.18867627117275357]                                                          
,                                                                                                                      
[96.44655446643753, 0.3563660198951359, 0.18953548435347312]                                                           
,                                                                                                                      
[51.333542518787034, 0.38661525170019934

[74.06719073649789, 0.3416237062116673, 0.18305592396646966]                                                           
,                                                                                                                      
[65.17846980099736, 0.31450335654981626, 0.1833166273215279]                                                           
,                                                                                                                      
[48.80152054410214, 0.356256331749606, 0.18699347814409176]                                                            
,                                                                                                                      
[67.77715438744684, 0.2866145290714553, 0.1939323451421259]                                                            
,                                                                                                                      
[68.23686483074736, 0.24827855054448678,

In [1]:
array = [[27.149627715381794, 0.1858475031658863, 0.1797702155028539]                                                           
,                                                                                                                      
[67.0080586166762, 0.01614439296735901, 0.18812390955620187]                                                           
,                                                                                                                      
[99.60546834896662, 0.25492718310042795, 0.18867627117275357]                                                          
,                                                                                                                      
[96.44655446643753, 0.3563660198951359, 0.18953548435347312]                                                           
,                                                                                                                      
[51.333542518787034, 0.38661525170019934, 0.18626469210305188]                                                         
,                                                                                                                      
[55.79394319828495, 0.34683018602923144, 0.19415383859083674]                                                          
,                                                                                                                      
[45.536384417304696, 0.22141984564698214, 0.19119228645647596]                                                         
,                                                                                                                      
[15.874495893867104, 0.22108059092751775, 0.17079970236165828]                                                         
,                                                                                                                      
[77.87303604183835, 0.29857908428855895, 0.19764429922456653]                                                          
,                                                                                                                      
[19.91545058969448, 0.056575973259107855, 0.17682736553871048]                                                         
,                                                                                                                      
[71.46932333394855, 0.11092092892646917, 0.18741114826955604]                                                          
,                                                                                                                      
[80.99984041891527, 0.1927937651503583, 0.185222386395905]                                                             
,                                                                                                                      
[65.00626992078715, 0.32940161209549773, 0.1920483577626807]                                                           
,                                                                                                                      
[32.40031075543457, 0.20648363902137734, 0.1852226072995795]                                                           
,                                                                                                                      
[4.820143373827114, 0.0669839217252051, 0.1501239887646216]                                                            
,                                                                                                                      
[64.69449940456917, 0.1902442246652443, 0.18898157410364197]                                                           
,                                                                                                                      
[11.230506231799549, 0.012194796014974293, 0.16853671013984786]                                                        
,                                                                                                                      
[50.01692354997038, 0.2938967419488821, 0.18424445515483853]                                                           
,                                                                                                                      
[8.632047711747326, 0.17110596522832952, 0.1588277057015962]                                                           
,                                                                                                                      
[18.73992761943543, 0.22117102396908406, 0.17498638773898997]                                                          
,                                                                                                                      
[85.55428656246627, 0.2924739449667694, 0.1842809383349268]                                                            
,                                                                                                                      
[84.31208802647878, 0.3521169892024472, 0.18386112886431316]                                                           
,                                                                                                                      
[57.96859015427981, 0.39148295722479615, 0.18590345556450713]                                                          
,                                                                                                                      
[39.98949190989167, 0.294834068664853, 0.1863445926474062]                                                             
,                                                                                                                      
[76.64431963533207, 0.25808757137108174, 0.19059735339620998]                                                          
,                                                                                                                      
[58.721942216328415, 0.32779023143470204, 0.1812598726355104]                                                          
,                                                                                                                      
[93.48988087081389, 0.26215698634049195, 0.1884324126708691]                                                           
,                                                                                                                      
[90.11410816242683, 0.13723593846445759, 0.18407976669338424]                                                          
,                                                                                                                      
[33.3273513817813, 0.39899742403166666, 0.17889729829622267]                                                           
,                                                                                                                      
[73.8854951641791, 0.3558742887404363, 0.1933979710146121]                                                             
,                                                                                                                      
[58.33206991815464, 0.3216155755882389, 0.1880754532944916]                                                            
,                                                                                                                      
[69.9036010786182, 0.2754082937498932, 0.19055975684396312]                                                            
,                                                                                                                      
[99.95161700134523, 0.3691100265436079, 0.18557148179202648]                                                           
,                                                                                                                      
[42.45714094582632, 0.31342457345965996, 0.18338056025688518]                                                          
,                                                                                                                      
[49.45522973325475, 0.2417053618068758, 0.18838680365608362]                                                           
,                                                                                                                      
[78.08775384504523, 0.34484915946247063, 0.18843816956827678]                                                          
,                                                                                                                      
[27.98410766829446, 0.3761166269352798, 0.18200179593774557]                                                           
,                                                                                                                      
[55.01343507167974, 0.24737834507025752, 0.1864765587900107]                                                           
,                                                                                                                      
[63.146761709542716, 0.3065154824980237, 0.18924144388121542]                                                          
,                                                                                                                      
[37.9504847513528, 0.27902112011361724, 0.18916363611926498]                                                           
,                                                                                                                      
[93.06870981952588, 0.15660295332006802, 0.18702204051442295]                                                          
,                                                                                                                      
[45.58545064879211, 0.39913718420842603, 0.18704560892780722]                                                          
,                                                                                                                      
[68.3072126059531, 0.34456244279570686, 0.18920398786087916]                                                           
,                                                                                                                      
[84.44748962961339, 0.37454748372153346, 0.1873004716248999]                                                           
,                                                                                                                      
[61.92701129842643, 0.033666606239885655, 0.1875327880592061]                                                          
,                                                                                                                      
[53.80152567205123, 0.22716708274661385, 0.19204683668702308]                                                          
,                                                                                                                      
[71.85290153059671, 0.11376568073546278, 0.18734582290562565]                                                          
,                                                                                                                      
[23.75465938606854, 0.20710962038129027, 0.18554390415592487]                                                          
,                                                                                                                      
[1.0921545013925211, 0.2758438402153285, 0.142416206550997]                                                            
,                                                                                                                      
[87.81734121227724, 0.3367847917749934, 0.1886856290366324]                                                            
,                                                                                                                      
[46.88051513857816, 0.2350813420121174, 0.1859513352897044]                                                            
,                                                                                                                      
[81.83716084071942, 0.3073600276677657, 0.1890358286018565]                                                            
,                                                                                                                      
[75.90454210156899, 0.17117412252961048, 0.19044590905941236]                                                          
,                                                                                                                      
[66.2068716197665, 0.3656386934385627, 0.18831289972268353]                                                            
,                                                                                                                      
[35.90120256356344, 0.2040805140038438, 0.18304170685840193]                                                           
,                                                                                                                      
[95.21514274285708, 0.3864847161603536, 0.18487329231821292]                                                           
,                                                                                                                      
[79.07003453251801, 0.28759682115378676, 0.18744811019418153]                                                          
,                                                                                                                      
[27.385358417178914, 0.08435296436176416, 0.18226655382643914]                                                         
,                                                                                                                      
[97.95771080320738, 0.2602521682864981, 0.18418868936875213]                                                           
,                                                                                                                      
[42.26794247958934, 0.32126298305158224, 0.1799734425208088]                                                           
,                                                                                                                      
[52.561826931587575, 0.18033574872665356, 0.18736334410925665]                                                         
,                                                                                                                      
[61.021947928395804, 0.13736626264135587, 0.1846918047059785]                                                          
,                                                                                                                      
[11.924196968865395, 0.26883841233892114, 0.16934987713959035]                                                         
,                                                                                                                      
[89.32099397245261, 0.29982990802378073, 0.1923608350317866]                                                           
,                                                                                                                      
[71.91988563476565, 0.38551146644485246, 0.18436361979390734]                                                          
,                                                                                                                      
[56.20598139936723, 0.3559657430948904, 0.1834278887855863]                                                            
,                                                                                                                      
[72.68718018439259, 0.3294259441500727, 0.19125148087172106]                                                           
,                                                                                                                      
[82.27236564362202, 0.3620883551253834, 0.1787673311054053]                                                            
,                                                                                                                      
[74.06719073649789, 0.3416237062116673, 0.18305592396646966]                                                           
,                                                                                                                      
[65.17846980099736, 0.31450335654981626, 0.1833166273215279]                                                           
,                                                                                                                      
[48.80152054410214, 0.356256331749606, 0.18699347814409176]                                                            
,                                                                                                                      
[67.77715438744684, 0.2866145290714553, 0.1939323451421259]                                                            
,                                                                                                                      
[68.23686483074736, 0.24827855054448678, 0.1908984989294448]                                                           
,                                                                                                                      
[60.53733388458365, 0.28655762854383143, 0.18410227926052772]                                                          
,                                                                                                                      
[79.02591129360532, 0.23285717514690757, 0.1817538792607467]                                                           
,                                                                                                                      
[30.639563548200396, 0.3002732588831247, 0.1847427108302524]                                                           
,                                                                                                                      
[51.51107623504122, 0.21030913061443757, 0.18606081294292626]                                                          
,                                                                                                                      
[56.93008737837159, 0.3310643195827676, 0.187427984879066]                                                             
,                                                                                                                      
[92.04997279256324, 0.21487333471888018, 0.18822378263684889]                                                          
,                                                                                                                      
[42.48625580178252, 0.2701366303066361, 0.18970575060965314]                                                           
,                                                                                                                      
[74.9839930435807, 0.19842456751996446, 0.192672583313159]                                                             
,                                                                                                                      
[86.19399165187552, 0.282414204952815, 0.1949852589378465]                                                             
,                                                                                                                      
[86.41234586529345, 0.25249495695369034, 0.1865811051303358]                                                           
,                                                                                                                      
[99.96148210334323, 0.31485366102377, 0.187698535287987]                                                               
,                                                                                                                      
[83.25543008375266, 0.37533977136895885, 0.18340944617771648]                                                          
,                                                                                                                      
[91.42742258949235, 0.22072452030705217, 0.1897842806010573]                                                           
,                                                                                                                      
[95.7079549530274, 0.3488232081354411, 0.18561829483907086]                                                            
,                                                                                                                      
[70.43513473922557, 0.39506888940398227, 0.1860132757236275]                                                           
,                                                                                                                      
[77.57069130753457, 0.23674338506534393, 0.1884111744612487]                                                           
,                                                                                                                      
[87.68069301771736, 0.1845037207762958, 0.18641458214144624]                                                           
,                                                                                                                      
[22.921078467334738, 0.2666133543516436, 0.18429067621249942]                                                          
,                                                                                                                      
[97.47286175293382, 0.3357369734639984, 0.1946640414841083]                                                            
,                                                                                                                      
[99.11946549136306, 0.14921165391140664, 0.18628795597615597]                                                          
,                                                                                                                      
[94.21112875948975, 0.28197116501730707, 0.1858027480613122]                                                           
,                                                                                                                      
[97.5731000727608, 0.3361935340162743, 0.19067309381538206]                                                            
,                                                                                                                      
[85.55548074644317, 0.3042368982434653, 0.1905968545561758]                                                            
,                                                                                                                      
[79.80987462552552, 0.3235973567881322, 0.18716869565235017]                                                           
,                                                                                                                      
[90.23715955463355, 0.24319987827458656, 0.19131968275001238]                                                          
,                                                                                                                      
[88.44316497915189, 0.25887307356503886, 0.1819804959916483]                                                           
,                                                                                                                      
[80.46503059251364, 0.2930650921840194, 0.1843685873867615] ]

In [5]:
metric = pd.DataFrame(array)
metric.columns = ['features','reg','MRR']
metric.head(5)

,features,reg,MRR
0,27.149628,0.185848,0.179770
1,67.008059,0.016144,0.188124
2,99.605468,0.254927,0.188676
3,96.446554,0.356366,0.189535
4,51.333543,0.386615,0.186265


In [6]:
metric.sort_values(by=['MRR'],ascending=False).head(10)

,features,reg,MRR
8,77.873036,0.298579,0.197644
81,86.193992,0.282414,0.194985
91,97.472862,0.335737,0.194664
5,55.793943,0.346830,0.194154
71,67.777154,0.286615,0.193932
29,73.885495,0.355874,0.193398
80,74.983993,0.198425,0.192673
63,89.320994,0.299830,0.192361
12,65.006270,0.329402,0.192048
45,53.801526,0.227167,0.192047


In [7]:
metric.to_csv('metric_MRR.csv', index=False)

In [15]:
%matplotlib qt
import matplotlib.pyplot as plt
from matplotlib import cm
colormap = cm.hot
x = metric['reg']
y = metric['features'] 
c = metric['MRR']


fig = plt.figure()
img = plt.scatter(x, y, c=c, cmap=plt.jet())
plt.xlabel('reg')
plt.ylabel('features')
fig.colorbar(img)
plt.show()